# CS-6570 Lecture 5 - Data Wrangling

**Dylan Zwick**

*Weber State University*

Today, we're going to go over a few more features and functions that Pandas offers for manipulating and exploring data - sometimes called "Data Wrangling".

To do this, let's load up some data. We'll use the fortune500 dataset from Lecture 2.

In [1]:
import numpy as np
import pandas as pd

In [13]:
fortune500_df = pd.read_csv('Datasets/fortune500.csv')
fortune500_df.columns = ['year', 'rank', 'company', 'revenue', 'profit']
fortune500_df

,year,rank,company,revenue,profit
0,1955,1,General Motors,9823.5,806
1,1955,2,Exxon Mobil,5661.4,584.8
2,1955,3,U.S. Steel,3250.4,195.4
3,1955,4,General Electric,2959.1,212.6
4,1955,5,Esmark,2510.8,19.1
...,...,...,...,...,...
25495,2005,496,Wm. Wrigley Jr.,3648.6,493
25496,2005,497,Peabody Energy,3631.6,175.4
25497,2005,498,Wendy's International,3630.4,57.8
25498,2005,499,Kindred Healthcare,3616.6,70.6


We'll start by dropping any row that contains an N.A. for profit. You can check Lecture 2 for specifics of how we did this for this dataframe.

Rob: This gives the rows that don't contain numberish characters and then we look for the inverse of that with the ~ operator to get the rows that are still good. This provides a boolean mask that contain true/false values. 

In [14]:
non_numeric_profits = fortune500_df.profit.str.contains('[^0-9.-]')
fortune500_df = fortune500_df.loc[~non_numeric_profits]
# Including the ~ gives the not [true]
fortune500_df

,year,rank,company,revenue,profit
0,1955,1,General Motors,9823.5,806
1,1955,2,Exxon Mobil,5661.4,584.8
2,1955,3,U.S. Steel,3250.4,195.4
3,1955,4,General Electric,2959.1,212.6
4,1955,5,Esmark,2510.8,19.1
...,...,...,...,...,...
25495,2005,496,Wm. Wrigley Jr.,3648.6,493
25496,2005,497,Peabody Energy,3631.6,175.4
25497,2005,498,Wendy's International,3630.4,57.8
25498,2005,499,Kindred Healthcare,3616.6,70.6


We note that there are now 25,131 rows, but the index still goes from 0 to 25499. In other words, the index is no longer sequential, but contains gaps.

If we want to get rid of those gaps - reset the index - we can do so with the following:

Rob: Note how the old index column is still maintained

In [16]:
fortune500_df.reset_index(inplace = True)
fortune500_df

,level_0,index,year,rank,company,revenue,profit
0,0,0,1955,1,General Motors,9823.5,806
1,1,1,1955,2,Exxon Mobil,5661.4,584.8
2,2,2,1955,3,U.S. Steel,3250.4,195.4
3,3,3,1955,4,General Electric,2959.1,212.6
4,4,4,1955,5,Esmark,2510.8,19.1
...,...,...,...,...,...,...,...
25126,25126,25495,2005,496,Wm. Wrigley Jr.,3648.6,493
25127,25127,25496,2005,497,Peabody Energy,3631.6,175.4
25128,25128,25497,2005,498,Wendy's International,3630.4,57.8
25129,25129,25498,2005,499,Kindred Healthcare,3616.6,70.6


Rob: The following is important

Note that we've now created a new index, but we've also created a column "index" that retains the previous index values. So, you don't lose the old index values when you reset.

Sometimes this is great. Other times, it's just another column, and you should probably drop it:

In [17]:
fortune500_df.drop(columns = ['index'], inplace = True)
fortune500_df

/var/folders/vw/9gqqlzfx5yq6pxkdr2xp20hm0000gn/T/ipykernel_2205/788303630.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fortune500_df.drop(columns = ['index'], inplace = True)


,level_0,year,rank,company,revenue,profit
0,0,1955,1,General Motors,9823.5,806
1,1,1955,2,Exxon Mobil,5661.4,584.8
2,2,1955,3,U.S. Steel,3250.4,195.4
3,3,1955,4,General Electric,2959.1,212.6
4,4,1955,5,Esmark,2510.8,19.1
...,...,...,...,...,...,...
25126,25126,2005,496,Wm. Wrigley Jr.,3648.6,493
25127,25127,2005,497,Peabody Energy,3631.6,175.4
25128,25128,2005,498,Wendy's International,3630.4,57.8
25129,25129,2005,499,Kindred Healthcare,3616.6,70.6


Now, suppose we wanted to aggregate the values in the dataframe above into individual years. The way we'd do this for, say, revenue is straightforward, but for the companies it's a bit tricky. One way you could do it is by collapsing them all into a list.

Rob: We're adding two additional columns to maintain the list of companies who were in the data by year. Cool example. Could also have been done with a pivot table

In [20]:
companies_by_year_df = fortune500_df.groupby(by = ['year']).agg(
    company_list = ('company',lambda x: x.tolist()), total_revenue = ('revenue', np.sum))
companies_by_year_df

,company_list,total_revenue
year,,
1955,"[General Motors, Exxon Mobil, U.S. Steel, Gene...",136222.7
1956,"[General Motors, Exxon Mobil, Ford Motor, U.S....",161155.0
1957,"[General Motors, Exxon Mobil, Ford Motor, U.S....",173968.7
1958,"[General Motors, Exxon Mobil, Ford Motor, U.S....",187854.7
1959,"[General Motors, Exxon Mobil, Ford Motor, Gene...",176549.0
1960,"[General Motors, Exxon Mobil, Ford Motor, Gene...",197220.7
1961,"[General Motors, Exxon Mobil, Ford Motor, Gene...",204455.1
1962,"[General Motors, Exxon Mobil, Ford Motor, Gene...",208939.3
1963,"[General Motors, Exxon Mobil, Ford Motor, Gene...",229082.1


From here, suppose we wanted to check out the year-over-year revenue change. One way we could do this is by using the "shift" function:

Rob: Super clever function to keep the index the same, but move everything a row down. The dataframe in memory remains the same, it's just a view that is changing.

In [24]:
companies_by_year_df.shift(1)

,company_list,total_revenue,change_in_revenue
year,,,
1955,None,NaN,NaN
1956,"[General Motors, Exxon Mobil, U.S. Steel, Gene...",136222.7,NaN
1957,"[General Motors, Exxon Mobil, Ford Motor, U.S....",161155.0,24932.3
1958,"[General Motors, Exxon Mobil, Ford Motor, U.S....",173968.7,12813.7
1959,"[General Motors, Exxon Mobil, Ford Motor, U.S....",187854.7,13886.0
1960,"[General Motors, Exxon Mobil, Ford Motor, Gene...",176549.0,-11305.7
1961,"[General Motors, Exxon Mobil, Ford Motor, Gene...",197220.7,20671.7
1962,"[General Motors, Exxon Mobil, Ford Motor, Gene...",204455.1,7234.4
1963,"[General Motors, Exxon Mobil, Ford Motor, Gene...",208939.3,4484.2


In [25]:
companies_by_year_df['change_in_revenue'] = (companies_by_year_df['total_revenue'] 
                                             - companies_by_year_df.shift(1)['total_revenue'])
companies_by_year_df

,company_list,total_revenue,change_in_revenue
year,,,
1955,"[General Motors, Exxon Mobil, U.S. Steel, Gene...",136222.7,NaN
1956,"[General Motors, Exxon Mobil, Ford Motor, U.S....",161155.0,24932.3
1957,"[General Motors, Exxon Mobil, Ford Motor, U.S....",173968.7,12813.7
1958,"[General Motors, Exxon Mobil, Ford Motor, U.S....",187854.7,13886.0
1959,"[General Motors, Exxon Mobil, Ford Motor, Gene...",176549.0,-11305.7
1960,"[General Motors, Exxon Mobil, Ford Motor, Gene...",197220.7,20671.7
1961,"[General Motors, Exxon Mobil, Ford Motor, Gene...",204455.1,7234.4
1962,"[General Motors, Exxon Mobil, Ford Motor, Gene...",208939.3,4484.2
1963,"[General Motors, Exxon Mobil, Ford Motor, Gene...",229082.1,20142.8


If we want to view the years with the most negative to most positive changes in revenue, we can do so with the sort_values function.

In [35]:
# Rob: It doesn't like absolute value in the 'by' operator because that column doesn't exist
companies_by_year_df.sort_values(by=['change_in_revenue'], ascending=False)

,company_list,total_revenue,change_in_revenue
year,,,
1995,"[General Motors, Ford Motor, Exxon Mobil, Wal-...",4247422.7,1902349.4
2001,"[Exxon Mobil, Wal-Mart Stores, General Motors,...",7155910.3,855777.2
2005,"[Wal-Mart Stores, Exxon Mobil, General Motors,...",8244202.6,778934.1
2000,"[General Motors, Wal-Mart Stores, Exxon Mobil,...",6300133.1,578253.8
2004,"[Wal-Mart Stores, Exxon Mobil, General Motors,...",7465268.5,537496.5
1998,"[General Motors, Ford Motor, Exxon Mobil, Wal-...",5503123.6,454731.6
1996,"[General Motors, Ford Motor, Exxon Mobil, Wal-...",4665242.4,417819.7
1997,"[General Motors, Ford Motor, Exxon Mobil, Wal-...",5048392.0,383149.6
1980,"[Exxon Mobil, General Motors, Mobil, Ford Moto...",1433378.0,227525.1


If we want to unravel our list of companies, we can do so with the "explode" function, which takes a list and returns a bunch of rows, one for each element in the list.

In [36]:
# Rob: The explode() function will create a new row for every element in the list
companies_by_year_explode_df = companies_by_year_df.explode(column = ['company_list'])
companies_by_year_explode_df

,company_list,total_revenue,change_in_revenue
year,,,
1955,General Motors,136222.7,NaN
1955,Exxon Mobil,136222.7,NaN
1955,U.S. Steel,136222.7,NaN
1955,General Electric,136222.7,NaN
1955,Esmark,136222.7,NaN
...,...,...,...
2005,Wm. Wrigley Jr.,8244202.6,778934.1
2005,Peabody Energy,8244202.6,778934.1
2005,Wendy's International,8244202.6,778934.1


Here if we wanted to have a unique index we could get one by resetting the index, but this is a nice example of where you might want to hold onto the original index as well.

In [39]:
companies_by_year_explode_df = companies_by_year_explode_df.reset_index()
companies_by_year_explode_df

,index,year,company_list
0,0,1955,General Motors
1,1,1955,Exxon Mobil
2,2,1955,U.S. Steel
3,3,1955,General Electric
4,4,1955,Esmark
...,...,...,...
25126,25126,2005,Wm. Wrigley Jr.
25127,25127,2005,Peabody Energy
25128,25128,2005,Wendy's International
25129,25129,2005,Kindred Healthcare


In [42]:
# Rob: Could drop the columns we don't want or we can just mark the columns we want to keep
# Rob: If you have a CSV file with a list of companies that will almost certainly be read in as string and not a Python list. 
# You must convert to a column of lists
companies_by_year_explode_df = companies_by_year_explode_df[['year','company_list']]
companies_by_year_explode_df

,year,company_list
0,1955,General Motors
1,1955,Exxon Mobil
2,1955,U.S. Steel
3,1955,General Electric
4,1955,Esmark
...,...,...
25126,2005,Wm. Wrigley Jr.
25127,2005,Peabody Energy
25128,2005,Wendy's International
25129,2005,Kindred Healthcare


### Merging

If you're familiar with SQL, you're probably got some experience merging or "joining" two different tables along a key or set of keys. Well, you can do the same thing in Pandas.

Let's start with a simple example:

In [43]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"], "data1": pd.Series(range(7), dtype = "Int64")})
df2 = pd.DataFrame({"key": ["a", "b", "d"], "data2": pd.Series(range(3), dtype = "Int64")})

In [44]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [45]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


If we merge these, we get:

In [48]:
# Rob: Merge is the Pandas version of a SQL join
pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Rob: **Note that I didn't specify which column to join on. If that information is not specified, merge uses the overlapping column names as the keys if it can. It's good practice to specify explicitly - in this case both tables have the key column though:**

In [59]:
pd.merge(df1,df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


If the column names are different in each column you can specify them separately. Rob: Here the fundamental data is the same, but the key column is different

In [60]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"], "data1": pd.Series(range(7), dtype = "Int64")})
df4 = pd.DataFrame({"rkey": ["a", "b", "d"], "data2": pd.Series(range(3), dtype = "Int64")})

In [66]:
# Rob: With different key names, the default join throws an error
pd.merge(df3,df4)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [67]:
# Left and right inner joins are also possible using different key names
pd.merge(df3,df4,left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [68]:
# Left and right inner joins are also possible using different key names
pd.merge(df3,df4,left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [69]:
# Left and right inner joins are also possible using different key names
pd.merge(df3,df4,left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [70]:
# Left and right inner joins are also possible using different key names
pd.merge(df3,df4,left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


You may notice the "c" and "d" values and associated data are missing from the result. This is because by default Pandas, like SQL,  does an "inner" join; the keys in the result are the intersection of both tables.

The types of possible joins are:

* "inner" - Use only the key combinations observed in both tables.
* "left" - Use all key combinations found in the left table.
* "right" - Use all key combinations found in the right table.
* "outer" - Use all key combinations found in both tables together.

You can specify the type of join to use with the "how" parameter in a merge call.

In [71]:
pd.merge(df1,df2, on='key', how="inner")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [72]:
pd.merge(df1,df2, on='key', how="left")

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,c,3,<NA>
4,a,4,0
5,a,5,0
6,b,6,1


In [73]:
pd.merge(df1,df2, on='key', how="right")

,key,data1,data2
0,a,2,0
1,a,4,0
2,a,5,0
3,b,0,1
4,b,1,1
5,b,6,1
6,d,<NA>,2


In [74]:
pd.merge(df1,df2, on='key', how="outer")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0
6,c,3,<NA>
7,d,<NA>,2


Rob: In the above example, rows 6 and 7 are in the each of the dataframes.